# Introduction to Sessions
In this notebook, we'll get used to working with sessions in Azure Quantum by using a session to run multiple Qiskit jobs on a target.

## What is a session?
A session is a logical grouping of one or more jobs submitted to a single target (backend). Each session has a unique ID that is attached to all jobs in that session. While sessions can be used broadly in Azure Quantum, they are particularly helpful for complex hybrid algorithms where classical code is interwoven with a large number of quantum jobs. With sessions you can focus on your algorithm as a whole instead of managing individual quantum jobs.

We'll use Qiskit to submit jobs to the IonQ simulator backend for this example, but you can use sessions with Q# + Python and Cirq, and variety of backends. See the [sessions documentation](https://aka.ms/AQ/Hybrid/Sessions/Docs) for details.

### 1. Connect to Azure Quantum and build a quantum circuit

Before we create a session, we'll construct an instance of the `AzureQuantumProvider` and select a backend.

In [ ]:
# Connect to the Azure Quantum workspace
from qiskit import QuantumCircuit
from qiskit.tools.monitor import job_monitor
from azure.quantum.qiskit import AzureQuantumProvider

provider = AzureQuantumProvider (
            resource_id = "",
            location = "")

ionq_sim = provider.get_backend('ionq.simulator')
quantinuum_sim = provider.get_backend('quantinuum.sim.h1-1e')
rigetti_sim = provider.get_backend('rigetti.sim.qvm')

# Set the backend you want to use here.
# WARNING: Quantinuum simulator usage is not unlimited. Running this sample against it could consume a significant amount of your eHQC quota.
backend = ionq_sim

For this example, we'll create a simple 2-qubit circuit that we will run multiple times in a session. 

In [ ]:
# Create a quantum circuit acting on two qubits
circuit = QuantumCircuit(2, 2)
circuit.name = "GenerateRandomBit"
circuit.h(0)
circuit.cnot(0,1)
circuit.measure([0,1], [0,1])

# Print out the circuit
circuit.draw()

### 2. Run quantum jobs in a session
Now it's time to create a session! We open a session using a ```with``` statement so the session is automatically closed upon completion of our program. Then we run as many individual quantum jobs within the session as we need to, in this case we'll submit our circuit three times. You can also add classical code between individual quantum job submissions within the session.

In [ ]:
with backend.open_session(name="Qiskit Session") as session:
    job1 = backend.run(circuit=circuit, shots=100, job_name="Job 1") # First job submission
    job_monitor(job1)
    # Classical code could go here
    job2 = backend.run(circuit=circuit, shots=100, job_name="Job 2") # Second job submission
    job_monitor(job2)
    # Classical code could go here
    job3 = backend.run(circuit=circuit, shots=100, job_name="Job 3") # Third job submission
    job_monitor(job3)

And that's it! You've run your first program with sessions! 🥳🎉🎊

Running our simple program within a session lets us better organize and manage the jobs. For example, we can list all jobs in the session:

In [ ]:
session_jobs = session.list_jobs()
[session_job.details.name for session_job in session_jobs]

You also can view your session in the Job management blade of your Quantum Workspace in the Azure Portal. Instead of seeing individual jobs immediately, you will see the session instead, and can click on the session to drill down into the individual jobs. 

This may be more organization than we need for our simple 3 quantum jobs program, but the ability to view and manage jobs at the session level can be a big help as you start submitting multiple runs of complex hybrid algorithms that can countain hundreds or thousands of individual jobs.

### 3. Change a session's job failure policy

Many parameterized hybrid algorithms are sensitive to individual quantum job failures: if one job fails, the correct parameters for subsequent jobs may not be properly set. This can lead to situations where improperly parameterized jobs continue to be submitted to targets, wasting valuable time and resources.

To prevent this, job failures cause sessions to be closed by default. The Azure Quantum service attempts to cancel any pending jobs already submitted to the target, and new jobs submitted to the closed session are rejected.

If you'd like to continue submitting new jobs within a session even if a previous job fails, you can change this default behavior by specifying a job failure policy of ```job_failure_policy=SessionJobFailurePolicy.CONTINUE``` when creating the session:

```python
with backend.open_session(name="Qiskit Session", job_failure_policy=SessionJobFailurePolicy.CONTINUE) as session:
   job1 = backend.run(circuit=circuit, shots=100, job_name="Job 1") # First job submission
   ...
```

### 4. Next steps
You now have everything you need to get starting building quantum algorithms with sessions! As a next step, you can explore advanced samples in the hybrid quantum computing sample gallery, including an example of using sessions to run the Variational Quantum Eigensolver to estimate the ground state energy of a hydrogen molecule. As you create more complex algorithms with extended runtime, we recommend [running them locally](https://learn.microsoft.com/en-us/azure/quantum/how-to-long-running-experiments#local-development) to avoid job failures caused by occasional connectivity issues with Azure Quantum's hosted notebooks.

For more information on sessions, including how Quantinuum provides **reserved QPU access** once a session is started, check out the [sessions documentation](https://aka.ms/AQ/Hybrid/Sessions/Docs).

We can't wait to see what you build!

